In [ ]:
pip install langchain faiss-cpu beautifulsoup4 requests langchain-upstage

In [145]:
import os
import requests
from bs4 import BeautifulSoup
from langchain_upstage import UpstageEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from concurrent.futures import ThreadPoolExecutor

os.environ["UPSTAGE_API_KEY"] = "up_coecXafSJVG1v17EEZ3lxjFbZ8xcD"

embedding_model = None
vectorstore = None
upstage_llm = None
qa_chain = None

def initialize_models():
    global embedding_model, vectorstore, upstage_llm, qa_chain
    if embedding_model is None:
        embedding_model = UpstageEmbeddings(model="solar-embedding-1-large")
    if vectorstore is None:
        vectorstore = FAISS.from_texts(texts, embedding_model)
    if upstage_llm is None:
        upstage_llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))
    if qa_chain is None:
        qa_chain = load_qa_chain(llm=upstage_llm, chain_type="stuff")

initialize_models()

def extract_text_from_url(urls):
    all_texts = []

    def fetch_text(url):
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            paragraphs = soup.find_all('p')
            return "\n".join([para.get_text() for para in paragraphs])
        except Exception as e:
            print(f"Error processing {url}: {e}")
            return None

    with ThreadPoolExecutor() as executor:
        results = executor.map(fetch_text, urls)

    all_texts = [text for text in results if text]

    return all_texts

now_number = 28226
urls = []

for number in range(now_number, now_number-30, -1):
  urls.append("https://cse.knu.ac.kr/bbs/board.php?bo_table=sub5_1&wr_id=" + str(number))

document_text = extract_text_from_url(urls)

if isinstance(document_text, list):
    texts = []
    for doc in document_text:
        if isinstance(doc, str):
            texts.extend(text_splitter.split_text(doc))
        else:
            raise TypeError("Each document in the list must be a string")
else:
    raise TypeError("document_text must be a list of strings")

embedding_model = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

vectorstore = FAISS.from_texts(texts, embedding_model)

upstage_llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))

qa_chain = load_qa_chain(llm=upstage_llm, chain_type="stuff")

qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=vectorstore.as_retriever())

In [148]:
query = "가장 최근의 공지사항은 무엇인가요?"
print(qa.run(query))

가장 최근의 공지사항은 "2024년 2학기 대학원 학위논문 제출 대상자 중 연구윤리 교육 미수강신청 학생을 위한 최종 수강신청 일정 안내"입니다. 이 공지사항은 2024년 9월 24일에 게시되었습니다. 해당 학생은 2024년 9월 23일부터 9월 24일까지 신청해야 하며, 교육은 2024년 9월 25일부터 12월 23일까지 진행됩니다. 이번 학기 연구윤리 교육 이수 여부는 9월 말에 처리될 예정입니다.


In [147]:
query = "튜터 모집 기간은 언제인가요?"
print(qa.run(query))

튜터 모집 기간은 다음과 같습니다:

- 1차~5차 신청: 2024년 9월 19일(목) 오전까지 완료
- 6차 모집: 2024년 9월 23일(월) 10시까지

지원 방법은 구글 폼을 통해 신청하면 됩니다. 자세한 내용은 첨부된 파일을 확인해주세요.
